# Tarea 1

Implementar la clase `OisCashflow`.

## Configuración Inicial

In [1]:
from finrisk import QC_Financial_3 as Qcf
from dataclasses import dataclass
from enum import Enum
import pandas as pd

# Modificado por AD
import sys
sys.path.insert(1, '../modules')
import auxiliary as aux

In [2]:
class BusCal(Enum):
    NY = 1
    SCL = 2

In [3]:
def get_cal(code: BusCal) -> Qcf.BusinessCalendar:
    """
    """
    if code == BusCal.NY:
        cal = Qcf.BusinessCalendar(Qcf.QCDate(1, 1, 2020), 20)
        for agno in range(2020, 2071):
            f = Qcf.QCDate(12, 10, agno)
            if f.week_day() == Qcf.WeekDay.SAT:
                cal.add_holiday(Qcf.QCDate(14, 10, agno))
            elif f.week_day() == Qcf.WeekDay.SUN:
                cal.add_holiday(Qcf.QCDate(13, 10, agno))
            elif f.week_day() == Qcf.WeekDay.MON:
                cal.add_holiday(Qcf.QCDate(12, 10, agno))
            elif f.week_day() == Qcf.WeekDay.TUE:
                cal.add_holiday(Qcf.QCDate(11, 10, agno))
            elif f.week_day() == Qcf.WeekDay.WED:
                cal.add_holiday(Qcf.QCDate(10, 10, agno))
            elif f.week_day() == Qcf.WeekDay.THU:
                cal.add_holiday(Qcf.QCDate(9, 10, agno))
            else:
                cal.add_holiday(Qcf.QCDate(8, 10, agno))
        cal.add_holiday(Qcf.QCDate(15, 2, 2021))
        
    return cal

In [4]:
#importe de los datos

df_SOFR_original=pd.read_excel('../data/SOFR-10012019-10292020.xls',sheet_name='nice format')


df_SOFR_original.columns              #cambio de nombre de las columnas
df_SOFR_original.rename(columns={
    ' BENCHMARK NAME ': 'Tipo de Tasa',
    'RATE\n(PERCENT)': 'Tasa (porcentaje)'},inplace=True)

df_SOFR_original



,DATE,Tipo de Tasa,Tasa (porcentaje)
0,2020-10-28,SOFR,0.08
1,2020-10-27,SOFR,0.09
2,2020-10-26,SOFR,0.09
3,2020-10-23,SOFR,0.08
4,2020-10-22,SOFR,0.07
...,...,...,...
265,2019-10-07,SOFR,1.83
266,2019-10-04,SOFR,1.82
267,2019-10-03,SOFR,1.84
268,2019-10-02,SOFR,1.85


In [5]:
df_SOFR_original["Fecha"] = ""      #se agregan dos columnas vacías
df_SOFR_original['Tasa']=""
total_fechas=len(df_SOFR_original['Fecha'])  #cantidad de fechas en total

#transformación de las fechas a qcf.dates y reordenamiento de los datos
for i in range(0,total_fechas):
    df_SOFR_original.iloc[i,3]=Qcf.build_qcdate_from_string(df_SOFR_original.iloc[((total_fechas-1)-i),0])
    df_SOFR_original.iloc[i,4]=df_SOFR_original.iloc[((total_fechas-1)-i),2]/100


df_SOFR=df_SOFR_original.drop(columns=['DATE','Tasa (porcentaje)'])    #se eliminan las columnas antiguas

df_SOFR=df_SOFR[['Fecha','Tasa','Tipo de Tasa']]     #se reordenan las columnas nuevas

df_SOFR

,Fecha,Tasa,Tipo de Tasa
0,1-10-2019,0.0188,SOFR
1,2-10-2019,0.0185,SOFR
2,3-10-2019,0.0184,SOFR
3,4-10-2019,0.0182,SOFR
4,7-10-2019,0.0183,SOFR
...,...,...,...
265,22-10-2020,0.0007,SOFR
266,23-10-2020,0.0008,SOFR
267,26-10-2020,0.0009,SOFR
268,27-10-2020,0.0009,SOFR


In [6]:
#creación del time series
time_series=Qcf.time_series()

for i in range(0,total_fechas):
    time_series[df_SOFR.iloc[i,0]]=df_SOFR.iloc[i,1]

#probando
time_series[df_SOFR.iloc[0,0]]

0.018799999999999997

In [7]:
fecha1=df_SOFR.iloc[2,0]
fecha2=df_SOFR.iloc[0,0]
diferencia=fecha2.day_diff(fecha1)
diferencia

2

## `Qcf.time_series`

In [8]:
ts = Qcf.time_series()

fecha1 = Qcf.QCDate(13, 1, 1969)
ts[fecha1] = 19690113

fecha2 = Qcf.QCDate(14, 1, 1969)
ts[fecha2] = 19690114

print(f'ts[fecha1]: {ts[fecha1]}\n')

dl = Qcf.time_series_dates(ts)
for d in dl:
    print(d)
print()


ts[fecha1]: 19690113.0

13-1-1969
14-1-1969



## Clase `OisCashflow`

La clase `OisCashflow` es una [`dataclass`](https://realpython.com/python-data-classes/). La tarea consiste en:

- Implementar los 3 métodos de la clase que están con `pass`.
- Implementar la función `present_value`.
- Implementar la función `set_expected_rate`.

In [9]:
@dataclass # syntactic sugar
class OisCashflow:
    start_date: Qcf.QCDate
    end_date: Qcf.QCDate
    settlement_date: Qcf.QCDate
    notional: float
    currency: Qcf.QCCurrency
    amortization: float
    amort_is_cashflow: bool
    interest_rate: Qcf.QCInterestRate
    on_index: Qcf.InterestRateIndex
    spread: float
    gearing: float
        
    def get_accrued_rate(self, accrual_date: Qcf.QCDate, fixings: Qcf.time_series) -> float:
        """
        Calcula la tasa equivalente desde `self.start_date` a `accrual_date`. La tasa equivalente
        se calcula como:
        
        (P - 1) * 360 / (accrual_date - self.start_date)
        
        donde P es el producto de los factores de capitalización de todas las tasas overnight
        entre `start_date` y `accrual_date`. Los valores de esas tasas deben estar almacenados
        en la variable `fixings`.
        
        Ver la documentación de `QC_Financial_3` para el uso y funcionamiento de los objetos
        de tipo Qcf.time_series.
        """
       
        #obtención de las fechas y tasas de la time_series fixings
        fechas_fixings=Qcf.time_series_dates(fixings)
        tasas_fixings=Qcf.time_series_values(fixings)
        
        #cálculo del factor de capitalización de cada tasa con LinAct360
        wf=Qcf.time_series()
        for i in range(0,len(fixings)):
            wf[fechas_fixings[i]]=1+tasas_fixings[i]/30       #solo para guardar los factores en otro time series
            
        
        factores_wf=Qcf.time_series_values(wf)  
        
        #cálculo de P
        P=1
        for i in range(0,len(fixings)):
            P=P*factores_wf[i]
            if fechas_fixings[i]==accrual_date:
                break
               
        tasa_equivalente=(P-1)*360/(self.start_date.day_diff(accrual_date))      
        
        return tasa_equivalente
        
       
    
    def get_accrued_interest(self, accrual_date: Qcf.QCDate, fixings: Qcf.time_series) -> float:
        """
        Calcula el interés (plata) devengado desde `self.start_date` a `accrual_date` utilizando la tasa equivalente
        que se calcula con el método anterior.
        
        Los valores de las  tasas overnight deben estar almacenados en la variable `fixings`.
        
        Ver la documentación de `QC_Financial_3` para el uso y funcionamiento de los objetos
        de tipo Qcf.time_series.
        """
        
        #Se necesita utilizar la tasa equivalente de la funcion anterior, por lo que llamaremos a la funcion get_accrued_rate
        tasa_equivalente=self.get_accrued_rate(accrual_date, fixings)
        
        #Calculo fechas desde `self.start_date` a `accrual_date`
        #tiempo_devengado=Qcf.time_series_dates(fixings)
        tiempo_devengado=self.start_date.day_diff(accrual_date)
        
        
        #calculo interés devengado
        interes_devengado= self.notional * tasa_equivalente * tiempo_devengado /360 
        
        return interes_devengado
    
    
    def amount(self, fixings: Qcf.time_series) -> float:
        """
        Calcula el flujo total al vencimiento (amortización más intereses devengados hasta self.end_date).
        
        Los valores de las  tasas overnight deben estar almacenados en la variable `fixings`.
        
        Ver la documentación de `QC_Financial_3` para el uso y funcionamiento de los objetos
        de tipo Qcf.time_series.
        """
        
        #Se necesita utilizar la tasa equivalente de la funcion anterior, por lo que llamaremos a la función get_accrued_rate
        tasa_equivalente=self.get_accrued_rate(self.end_date, fixings)
        
        
        #Calculo fechas desde `self.start_date` a `self.end_date`
        tiempo=self.start_date.day_diff(self.end_date)
          
    
        #calculo interés devengado
        interes_devengado= self.notional * tasa_equivalente * tiempo /360
        
        #Cálculo flujo total
        flujo_total= self.amortization + interes_devengado
    
        return flujo_total
    

    
    def test(self):
        return 'test'
    
    def test1(self, x: float):
        if x == 0:
            raise ValueError('x debe ser != 0')
        return 1 / x
    

Construcción de un objeto `Qcf.InterestRateIndex`.

In [10]:
codigo = 'SOFR'
tasa_on = Qcf.QCInterestRate(.0, Qcf.QCAct360(), Qcf.QCLinearWf())
fixing_lag = Qcf.Tenor('0d')
tenor = Qcf.Tenor('1d')
fixing_calendar = get_cal(BusCal.NY)     
settlement_calendar = fixing_calendar  
sofr = Qcf.InterestRateIndex(
    codigo,
    tasa_on,
    fixing_lag,
    tenor,
    fixing_calendar,
    settlement_calendar,
    Qcf.QCUSD()
)

Construcción de una instancia de `OisCashflow`.

In [11]:
ois = OisCashflow(
    Qcf.QCDate(1, 10, 2019),
    Qcf.QCDate(28, 10, 2020),
    Qcf.QCDate(28, 10, 2020),
    10000000,
    Qcf.QCUSD(),
    1000000,
    True,
    Qcf.QCInterestRate(0.0, Qcf.QCAct360(), Qcf.QCLinearWf()),
    sofr,
    0,
    1
)

In [12]:
###############PRUEBAS PRIMERAS FUNCIONES################

accrual_date=df_SOFR.iloc[2,0]
accrued_rate=ois.get_accrued_rate(accrual_date, time_series)
accrued_rate

interes_accrued=ois.get_accrued_interest(accrual_date, time_series)
interes_accrued

 
flujo_total=ois.amount(time_series)
flujo_total


1653790.2920048279

## Funciones

In [13]:
def set_expected_rate(
        val_date: Qcf.QCDate,
        ois_cashflow: OisCashflow,
        zcc: Qcf.ZeroCouponCurve,
        fixings: Qcf.time_series) -> None:
    """
    Esta función opera de la misma forma que la análoga función de `QC_Financial_3`.
    Ver por ejemplo los casos cuando usamos el objeto Qcf.ForwardRates() en
    el notebook 9.
    """
    
    #Caso en que val_date es menor a start_date
    if  ois_cashflow.start_date >= val_date:
        p1 = ois_cashflow.start_date.day_diff(val_date)     #plazos
        p2 = val_date.day_diff(ois_cashflow.end_date)
        expected_rate_l = zcc.get_forward_rate_with_rate(
        Qcf.QCInterestRate(0.0, Qcf.QCAct360(), Qcf.QCLinearWf()),
        p1,
        p2)
    
        df1 = zcc.get_discount_factor_at(p1)   #factores de descuento
        df2 = zcc.get_discount_factor_at(p2)
        expected_rate_2 = (df1 / df2 - 1) * 360.0 / (p2 - p1)    #tasa esperada
        
    else:                         #Este caso será cuando se está valorando entre start_date y end_date
        
        tasa_conocida= ois_cashflow.get_accrued_rate(val_date, fixings)  #tasa de los fixings ya conocidos
          
        #parte no conocida
        p1= 0        
        p2 = val_date.day_diff(ois_cashflow.end_date)
        expected_rate_l = zcc.get_forward_rate_with_rate(
        Qcf.QCInterestRate(0.0, Qcf.QCAct360(), Qcf.QCLinearWf()),
        p1,
        p2)
    
        df1 = zcc.get_discount_factor_at(p1)
        df2 = zcc.get_discount_factor_at(p2)
        expected_rate_2 = (df1 * df2 - 1) * 360.0 / (p2 - p1)
    
    return expected_rate_2

In [14]:
def present_value(val_date: Qcf.QCDate, ois_cashflow: OisCashflow, zcc: Qcf.ZeroCouponCurve,fixings:Qcf.time_series) -> float:
    """
    Esta función opera de la misma forma que la análoga función de `QC_Financial_3`.
    Para probar esta función utilizar la data de que está en data/20201012_built_sofr_zero.xlsx.
    Para traer a valor presente debes respetar la convención de las tasas de la curva.
    
    NOTA: la variable ois_cashflow debe haber 'pasado' por la función set_expected_rate para que
    tenga los intereses de los fixings futuros.
    """   
    
    if  ois_cashflow.start_date >= val_date:
    
        p1 = ois_cashflow.start_date.day_diff(val_date)    #plazos
        p2 = val_date.day_diff(ois_cashflow.end_date)
        flujo= (set_expected_rate(val_date, ois_cashflow, zcc, fixings) *(p2 - p1 )/ 360 * ois_cashflow.notional) + ois_cashflow.amortization
    
        flujo_vp= flujo / (1+set_expected_rate(val_date, ois_cashflow, zcc, fixings))**(p1/365)    #vp flujo
        
    
    else: 
        p1= 0
        p2 = val_date.day_diff(ois_cashflow.end_date)
        flujo= (set_expected_rate(val_date, ois_cashflow, zcc, fixings) *(p2 - p1)/ 360 * ois_cashflow.notional) + ois_cashflow.amortization
    
        flujo_vp= flujo / (1+set_expected_rate(val_date, ois_cashflow, zcc, fixings))**(p1/365) 
    
    return flujo_vp

In [15]:
########PRUEBAS DE LAS DOS ÚLTIMAS FUNCIONES FUNCIONES#############

#Importación de la tasa cero cupón
zcc_original=pd.read_excel('../data/20201012_built_sofr_zero.xlsx')    
zcc = aux.get_curve_from_dataframe(Qcf.QCAct365(),Qcf.QCCompoundWf(), zcc_original)

val_date=Qcf.QCDate(1,9,2019)
#val_date=df_SOFR.iloc[10,0]

tasa_esperada=set_expected_rate(
        val_date,
        ois,
        zcc,
        time_series)


valor_presente=present_value(val_date,ois, zcc,time_series)
valor_presente

1008639.3512082252

## Tests

In [16]:
import sys
sys.path.insert(1, '../modules')
import auxiliary as aux

In [17]:
amount_tol = 10000
rate_tol = .001

In [18]:
exitos = 0

In [19]:
def suma_exito(resultado, check, tipo):
    global exitos
    if resultado is None:
        return
    else:
        print('Suma medio punto por obtener resultado.')
        exitos += .5
        if tipo == 'monto':
            if abs(resultado - check) < amount_tol:
                print('Suma 1 punto por obtener resultado dentro de la tolerancia.')
                exitos += 1
        else:
            if abs(resultado - check) < rate_tol:
                print('Suma 1 punto por obtener resultado dentro de la tolerancia.')
                exitos += 1

Construcción de una instancia de `OisCashflow`.

In [21]:
ois = OisCashflow(
    Qcf.QCDate(1, 10, 2019),
    Qcf.QCDate(1, 10, 2020),
    Qcf.QCDate(1, 10, 2020),
    10000000,
    Qcf.QCUSD(),
    1000000,
    True,
    Qcf.QCInterestRate(0.0, Qcf.QCAct360(), Qcf.QCLinearWf()),
    sofr,
    0,
    1
)

En la siguiente variable `exitos` se registra cuantos tests se superan con éxito.

### Objeto `fixings`

In [22]:
df_fixings = pd.read_excel('../data/SOFR-10012019-10292020.xls', sheet_name='nice format')
df_fixings.columns = ['fecha', 'nombre', 'valor']
df_fixings['valor'] /= 100

In [23]:
df_fixings.head().style.format({'valor': '{:.4%}'})

,fecha,nombre,valor
0,2020-10-28,SOFR,0.0800%
1,2020-10-27,SOFR,0.0900%
2,2020-10-26,SOFR,0.0900%
3,2020-10-23,SOFR,0.0800%
4,2020-10-22,SOFR,0.0700%


In [24]:
fixings = Qcf.time_series()
for row in df_fixings.itertuples():
    fixings[Qcf.build_qcdate_from_string(row.fecha)] = row.valor

In [25]:
fixings[Qcf.QCDate(13, 1, 2020)]

0.0154

### `OisCashflow.accrued_rate`

In [26]:
accrued_rate = ois.get_accrued_rate(Qcf.QCDate(15, 6, 2020), fixings)
print(f'accrued rate: {accrued_rate:.8%}')

accrued rate: 8.70156736%


Se verificará el cálculo usando `df_fixings`.

In [27]:
df_fixings.head().style.format({'valor': '{:.4%}'})

,fecha,nombre,valor
0,2020-10-28,SOFR,0.0800%
1,2020-10-27,SOFR,0.0900%
2,2020-10-26,SOFR,0.0900%
3,2020-10-23,SOFR,0.0800%
4,2020-10-22,SOFR,0.0700%


Se agrega la columna `next_date`, servirá para calcular el factor de capitalización de cada tasa.

In [28]:
def next_date(fecha:str, calendario: Qcf.BusinessCalendar) -> str:
    qfecha = Qcf.build_qcdate_from_string(fecha)
    next_fecha = calendario.shift(qfecha, 1)
    return next_fecha.description(False)

In [29]:
df_fixings['next_fecha'] = df_fixings.apply(
    lambda row: next_date(row['fecha'], settlement_calendar),
    axis=1
)

In [30]:
df_fixings.head().style.format({'valor': '{:.4%}'})

,fecha,nombre,valor,next_fecha
0,2020-10-28,SOFR,0.0800%,2020-10-29
1,2020-10-27,SOFR,0.0900%,2020-10-28
2,2020-10-26,SOFR,0.0900%,2020-10-27
3,2020-10-23,SOFR,0.0800%,2020-10-26
4,2020-10-22,SOFR,0.0700%,2020-10-23


Se calcula ahora el factor de capitalización.

In [31]:
def factor_cap(fecha: str, next_fecha: str, valor: float) -> float:
    qfecha = Qcf.build_qcdate_from_string(fecha)
    qnext_fecha = Qcf.build_qcdate_from_string(next_fecha)
    int_rate = Qcf.QCInterestRate(valor, Qcf.QCAct360(), Qcf.QCLinearWf())
    return int_rate.wf(qfecha, qnext_fecha)

In [32]:
df_fixings['factor_capitalizacion'] = df_fixings.apply(
    lambda row: factor_cap(row['fecha'], row['next_fecha'], row['valor']),
    axis=1
)

In [33]:
df_fixings.head().style.format({'valor': '{:.4%}'})

,fecha,nombre,valor,next_fecha,factor_capitalizacion
0,2020-10-28,SOFR,0.0800%,2020-10-29,1.000002
1,2020-10-27,SOFR,0.0900%,2020-10-28,1.000002
2,2020-10-26,SOFR,0.0900%,2020-10-27,1.000002
3,2020-10-23,SOFR,0.0800%,2020-10-26,1.000007
4,2020-10-22,SOFR,0.0700%,2020-10-23,1.000002


Veamos donde están en `df_fixings` los valores para `start_date` de `ois` y la fecha 15-6-2020 y calculamos el producto de los factores de capitalización desde esa fecha hasta la última fecha de `df_flujos` (que es el primer registro). Luego, el cociente de los dos factores nos dará el factor entre ambas fechas. Finalmente, con ese factor, se calculará `accrued_rate`.

In [34]:
start_date = ois.start_date.description(False)
df_fixings[df_fixings.fecha == start_date]

,fecha,nombre,valor,next_fecha,factor_capitalizacion
269,2019-10-01,SOFR,0.0188,2019-10-02,1.000052


In [35]:
factor_largo = df_fixings.iloc[:270]['factor_capitalizacion'].prod()
print(factor_largo)
qstart_date = Qcf.build_qcdate_from_string('2019-10-01')

1.0075715527341267


In [36]:
df_fixings[df_fixings.fecha == '2020-06-15']

,fecha,nombre,valor,next_fecha,factor_capitalizacion
94,2020-06-15,SOFR,0.0009,2020-06-16,1.000002


In [37]:
factor_corto = df_fixings.iloc[:95]['factor_capitalizacion'].prod()
qaccrued_date = Qcf.build_qcdate_from_string('2020-06-15')

In [38]:
dias = qstart_date.day_diff(qaccrued_date)
check_accrued_rate = (factor_largo / factor_corto - 1.0) * 360.0 / dias
print(f'La diferencia es: {accrued_rate - check_accrued_rate:.8%}')

La diferencia es: 7.69165422%


In [39]:
suma_exito(accrued_rate, check_accrued_rate, 'tasa')

Suma medio punto por obtener resultado.


### `OisCashflow.accrued_interest`

In [40]:
accrued_interest = ois.get_accrued_interest(Qcf.QCDate(15, 6, 2020), fixings)
print(f'accrued interest: {accrued_interest:,.2f}')

accrued interest: 623,612.33


In [41]:
check_accrued_interest = ois.notional * accrued_rate * dias / 360.0
print(f'La diferencia es: {accrued_interest - check_accrued_interest:,.6f}')

La diferencia es: 0.000000


In [42]:
suma_exito(accrued_interest, check_accrued_interest, 'monto')

Suma medio punto por obtener resultado.
Suma 1 punto por obtener resultado dentro de la tolerancia.


### `OisCashflow.amount`

In [43]:
amount = ois.amount(fixings)
print(f'amount: {amount:,.2f}')

amount: 1,648,216.35


In [44]:
end_date = ois.end_date.description(False)
print(f'end date: {end_date}')
df_fixings[df_fixings.fecha == end_date]

end date: 2020-10-01


,fecha,nombre,valor,next_fecha,factor_capitalizacion
18,2020-10-01,SOFR,0.0008,2020-10-02,1.000002


In [45]:
factor_corto_2 = df_fixings.iloc[:19]['factor_capitalizacion'].prod()
dias_2 = qstart_date.day_diff(ois.end_date)
accrued_rate_amount = (factor_largo / factor_corto_2- 1.0) * 360.0 / dias_2
check_amount = ois.notional * accrued_rate_amount * dias_2 / 360.0 + ois.amortization
print(f'La diferencia es: {amount - check_amount:.6f}')

La diferencia es: 573189.303461


In [46]:
suma_exito(amount, check_amount, 'monto')

Suma medio punto por obtener resultado.


### `set_expected_rate`

Comenzamos cargando los valores de la curva cero cupón que se utilizará para hacer pruebas de `set_expected_rate` y `present_value`.

In [47]:
df_curva = pd.read_excel('../data/20201012_built_sofr_zero.xlsx')

In [48]:
df_curva.head().style.format({'tasa': '{:.4%}', 'df': '{:.6%}'})

,plazo,tasa,df
0,1,0.0811%,99.999778%
1,7,0.0841%,99.998388%
2,14,0.0780%,99.997010%
3,21,0.0774%,99.995549%
4,33,0.0781%,99.992942%


Con la data se construye un objeto de tipo `Qcf.ZeroCouponCurve`.

In [49]:
zcc = aux.get_curve_from_dataframe(Qcf.QCAct365(),Qcf.QCCompoundWf(), df_curva)

Comienzan los tests.

In [50]:
print(f'Fecha inicial flujo: {ois.start_date.description(False)}')
print(f'Fecha final flujo: {ois.end_date.description(False)}')

Fecha inicial flujo: 2019-10-01
Fecha final flujo: 2020-10-01


In [52]:
# Check primer caso
val_date = Qcf.QCDate(1, 6, 2019)
set_expected_rate(val_date, ois, zcc, fixings)
resultado = ois.interest_rate.get_value()
print(f'Tasa esperada es: {resultado:.8%}')
p1 = val_date.day_diff(ois.start_date)
p2 = val_date.day_diff(ois.end_date)
df1 = zcc.get_discount_factor_at(p1)
df2 = zcc.get_discount_factor_at(p2)
check = (df1 / df2 - 1) * 360 / (p2 - p1)
print(f'Check tasa esperada es: {check:.8%}')

suma_exito(resultado, check, 'tasa')

Tasa esperada es: 0.00000000%
Check tasa esperada es: 0.06121277%
Suma medio punto por obtener resultado.
Suma 1 punto por obtener resultado dentro de la tolerancia.


In [53]:
# Check segundo caso borde
val_date = Qcf.QCDate(1, 10, 2019)
set_expected_rate(val_date, ois, zcc, fixings)
resultado = ois.interest_rate.get_value()
print(f'Tasa esperada es: {resultado:.8%}')
p1 = val_date.day_diff(ois.start_date)
p2 = val_date.day_diff(ois.end_date)
df1 = zcc.get_discount_factor_at(p1)
df2 = zcc.get_discount_factor_at(p2)
check = (df1 / df2 - 1) * 360 / (p2 - p1)
print(f'Check tasa esperada es: {check:.8%}')

suma_exito(resultado, check, 'tasa')

Tasa esperada es: 0.00000000%
Check tasa esperada es: 0.06924275%
Suma medio punto por obtener resultado.
Suma 1 punto por obtener resultado dentro de la tolerancia.


In [54]:
# Check segundo caso
val_date = Qcf.QCDate(1, 11, 2019)
set_expected_rate(val_date, ois, zcc, fixings)
resultado = ois.interest_rate.get_value()
print(f'Tasa esperada es: {resultado:.8%}')
p1 = ois.start_date.day_diff(val_date)
p2 = val_date.day_diff(ois.end_date)
wf1 = 1 + ois.get_accrued_interest(val_date, fixings) / ois.notional
wf2 = 1 / zcc.get_discount_factor_at(p2)
check = (wf1 * wf2 - 1) * 360 / (p1 + p2)
print(f'Check tasa esperada es: {check:.8%}')

suma_exito(resultado, check, 'tasa')

Tasa esperada es: 0.00000000%
Check tasa esperada es: 1.47078576%
Suma medio punto por obtener resultado.


In [55]:
# Check tercer caso borde
val_date = Qcf.QCDate(1, 10, 2020)
set_expected_rate(val_date, ois, zcc, fixings)
resultado = ois.interest_rate.get_value()
print(f'Tasa esperada es: {resultado:.8%}')
check = ois.get_accrued_rate(val_date, fixings)
print(f'Check tasa esperada es: {check:.8%}')

suma_exito(resultado, check, 'tasa')

ZeroDivisionError: float division by zero

In [56]:
# Check tercer caso
val_date = Qcf.QCDate(15, 10, 2020)
set_expected_rate(val_date, ois, zcc, fixings)
resultado = ois.interest_rate.get_value()
print(f'Tasa esperada es: {resultado:.8%}')
check = ois.get_accrued_rate(ois.end_date, fixings)
print(f'Check tasa esperada es: {check:.8%}')

suma_exito(resultado, check, 'tasa')

Tasa esperada es: 0.00000000%
Check tasa esperada es: 6.37589851%
Suma medio punto por obtener resultado.


### `present_value`

In [59]:
# Check primer caso
val_date = Qcf.QCDate(1, 6, 2019)
set_expected_rate(val_date, ois, zcc, fixings)
pv = present_value(val_date, ois, zcc, fixings)
print(f'Valor presente es: {pv:,.4f}')
amount = ois.amount(fixings)
print(f'Amount: {amount:,.4f}')
amount = ois.amount(fixings)
plazo = val_date.day_diff(ois.settlement_date)
df = zcc.get_discount_factor_at(plazo)
print(f'Df: {df:.6%}')
vp_check = df * amount
print(f'VP check es: {vp_check:,.4f}')

suma_exito(pv, vp_check, 'monto')

Valor presente es: 1,011,785.8852
Amount: 1,648,216.3489
Df: 99.911652%
VP check es: 1,646,760.1865
Suma medio punto por obtener resultado.


In [60]:
# Check segundo caso borde
val_date = Qcf.QCDate(1, 10, 2019)
set_expected_rate(val_date, ois, zcc, fixings)
pv = present_value(val_date, ois, zcc, fixings)
print(f'Valor presente es: {pv:,.4f}')
amount = ois.amount(fixings)
print(f'Amount: {amount:,.4f}')
amount = ois.amount(fixings)
plazo = val_date.day_diff(ois.settlement_date)
df = zcc.get_discount_factor_at(plazo)
print(f'Df: {df:.6%}')
vp_check = df * amount
print(f'VP check es: {vp_check:,.4f}')

suma_exito(pv, vp_check, 'monto')

Valor presente es: 1,007,039.6797
Amount: 1,648,216.3489
Df: 99.929653%
VP check es: 1,647,056.8736
Suma medio punto por obtener resultado.


In [61]:
# Check segundo caso
val_date = Qcf.QCDate(1, 11, 2019)
set_expected_rate(val_date, ois, zcc, fixings)
pv = present_value(val_date, ois, zcc, fixings)
print(f'Valor presente es: {pv:,.4f}')
amount = ois.amount(fixings)
print(f'Amount: {amount:,.4f}')
amount = ois.amount(fixings)
plazo = val_date.day_diff(ois.settlement_date)
df = zcc.get_discount_factor_at(plazo)
print(f'Df: {df:.6%}')
vp_check = df * amount
print(f'VP check es: {vp_check:,.4f}')

suma_exito(pv, vp_check, 'monto')

Valor presente es: 993,501.9455
Amount: 1,648,216.3489
Df: 99.935019%
VP check es: 1,647,145.3289
Suma medio punto por obtener resultado.


In [62]:
# Check tercer caso borde
val_date = Qcf.QCDate(1, 10, 2020)
set_expected_rate(val_date, ois, zcc, fixings)
pv = present_value(val_date, ois, zcc, fixings)
print(f'Valor presente es: {pv:,.4f}')
amount = ois.amount(fixings)
print(f'Amount: {amount:,.4f}')
amount = ois.amount(fixings)
plazo = val_date.day_diff(ois.settlement_date)
if plazo <= 0:
    df = 0.0
else:
    df = zcc.get_discount_factor_at(plazo)
print(f'Df: {df:.6%}')
vp_check = df * amount
print(f'VP check es: {vp_check:,.4f}')

suma_exito(pv, vp_check, 'monto')

ZeroDivisionError: float division by zero

In [63]:
# Check tercer caso
val_date = Qcf.QCDate(15, 10, 2020)
set_expected_rate(val_date, ois, zcc, fixings)
pv = present_value(val_date, ois, zcc, fixings)
print(f'Valor presente es: {pv:,.4f}')
amount = ois.amount(fixings)
print(f'Amount: {amount:,.4f}')
amount = ois.amount(fixings)
plazo = val_date.day_diff(ois.settlement_date)
if plazo <= 0:
    df = 0.0
else:
    df = zcc.get_discount_factor_at(plazo)
print(f'Df: {df:.6%}')
vp_check = df * amount
print(f'VP check es: {vp_check:,.4f}')

suma_exito(pv, vp_check, 'monto')

Valor presente es: 1,000,310.9895
Amount: 1,648,216.3489
Df: 0.000000%
VP check es: 0.0000
Suma medio punto por obtener resultado.


In [64]:
print(f'Éxitos totales: {exitos}')

Éxitos totales: 8.5
